In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from estudo import Estudo

dic = Estudo(
    ano = 2024,
    id_modulo = None,
    analito = None
).run()

In [2]:
erros = dic.get('erros')
len(erros)

370

In [3]:
modulos = sorted([int(i) for i in dic.keys() if i != 'erros'])
len(modulos)

291

In [5]:
dados = []
for mod in modulos:
    dados.extend(dic.get(mod)) 

In [52]:
cont = 0
clusters = 0
for data in dados:
    if len(i:=(data.query('identico=="Sim"'))) > 0:
        cont += len(i.cluster.unique())
    qtd_clusters = len(data.cluster.unique())
    clusters += qtd_clusters
cont, clusters

(284, 1570)

In [51]:
cont = 0
clusters = 0
for data in dados:
    if len(i:=(data.query('identico=="Não"'))) > 0:
        cont += len(i.cluster.unique())
    qtd_clusters = len(data.cluster.unique())
    clusters += qtd_clusters
cont, clusters

(1311, 1570)

In [ ]:
cont = 0
for data in dados:
    cont += (len(data.groupby('cluster', as_index=False).identico.agg(lambda x: len(set(x))).query('identico>1')))
    
cont


25

In [64]:
284+1311

1595

In [116]:
teste = pd.concat(dados)

### Número de grupos suspeitos por módulo (Top 10)


In [131]:
teste['grupos'] = teste['analito'] + ' -- ' + teste['cluster'].astype(str)
teste2 = teste.groupby(['id_modulo'], as_index=False).grupos.agg(lambda x: len(set(x))).sort_values('grupos', ascending=False).head(10)
teste2['id_modulo'] = teste2['id_modulo'].astype(str)

import plotly.express as px

px.bar(
    data_frame=teste2,
    x='grupos',
    y='id_modulo',
    title='Top 10 módulos com mais grupos suspeitos em colusão',
    text_auto=True
)

### 